In [ ]:
using Gmsh
using GridapGmsh
using Gridap
using Gridap.Geometry
using Gridap.TensorValues
using Gridap.CellData
using Gridap.ReferenceFEs
using Gridap.Fields
using LinearAlgebra
using LineSearches: BackTracking

In [ ]:

# Constants and model setup
const α = 10
const β = 1
const ls = 0.01
const η = 1e-15
const p = -0.5
const E1 = 1930.0
const E2 = 30.0
const v1 = 0.33
const v2 = 0.4
const GcI1 = 0.73
const σcI1 = 2445.42
const GcI2 = 0.28
const σcI2 = 50.0
model = GmshDiscreteModel("Circular_Hexagon_notch.msh")
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels, dimension)
brick = get_tag_from_name(labels, "Base")
mortar = get_tag_from_name(labels, "Lattice")


In [ ]:
# Material fields
function material_tag(tag)
    if tag == brick
        E =E1; ν = v1; GcI = GcI1; σcI = σcI1
    elseif tag == mortar
        E =E2; ν = v2; GcI = GcI2; σcI = σcI2
    end
    GcII = α * GcI
    σcII = β * σcI
    λ = E * ν / ((1 + ν) * (1 - 2 * ν))
    μ = E / (2 * (1 + ν))
    k = λ + μ
    ψ_Crit_I = σcI^2 / (2 * k)
    ψ_Crit_II = σcII^2 / (2 * μ)
    return λ, μ, k, GcI, GcII, ψ_Crit_I, ψ_Crit_II, σcI, σcII
end
function mI(tag)
    λ, μ, k, GcI, GcII, ψ_Crit_I, ψ_Crit_II, σcI, σcII= material_tag(tag)
    return 4 * GcI * k / (π * ls * σcI^2)
end

function mII(tag)
    λ, μ, k, GcI, GcII, ψ_Crit_I, ψ_Crit_II, σcI, σcII= material_tag(tag)
    return 4 * GcII * μ / (π * ls * σcII^2)
end


In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/2)*I4
I4_dev = I4_sym  - I4_vol  

In [ ]:
function degDer1(ϕ,tag)
    mIc = mI(tag)
    g = ((ϕ-1)*(ϕ*(2*p+1)+1)*mIc)/((ϕ*ϕ*(mIc*p+1)+(ϕ*(mIc-2)+1))^2) 
    return g
end

In [ ]:
function degDer2(ϕ,tag)
    mIIc = mII(tag)
    g = ((ϕ-1)*(ϕ*(2*p+1)+1)*mIIc)/((ϕ*ϕ*(mIIc*p+1)+(ϕ*(mIIc-2)+1))^2) 
    return g
end

In [ ]:
function σfun(ε,ε_in,s_in,tag)
    λ, μ, k, GcI, GcII, ψ_Crit_I, ψ_Crit_II, σcI, σcII= material_tag(tag)
    mIc = mI(tag)
    mIIc = mII(tag)
    εdev = I4_dev ⊙ ε
    σI = k*tr(ε)*one(ε)
    σII = 2*μ*εdev
    σ = ((1-s_in)^2 / ((1-s_in)^2 + ((mIc*s_in*(1 + p*s_in)))))*(σI) + ((1-s_in)^2 / ((1-s_in)^2 + ((mIIc*s_in*(1 + p*s_in)))))*(σII)
    return  σ
end  

In [ ]:
function Eigen(ε)
    εArray = get_array(ε)
    Λ, P = eigen(εArray)
    ε1 = Λ[1]
    ε2 = Λ[2]
    if ε1 >= 0 &&  ε2 >= 0
        Λpos = [ε1 0; 0 ε2]
        Λneg = [0 0; 0 0]
    elseif ε1 >= 0 &&  ε2 < 0
        Λpos = [ε1 0; 0 0]
        Λneg = [0 0; 0 ε2]
    elseif ε1 < 0 &&  ε2 >= 0
        Λpos = [0 0; 0 ε2]
        Λneg = [ε1 0; 0 0]
    elseif ε1 < 0 &&  ε2 < 0
        Λpos = [0 0; 0 0]
        Λneg = [ε1 0; 0 ε2]
    end
    εPos = TensorValue(P*Λpos*P')
    εNeg = TensorValue(P*Λneg*P')
    return εPos, εNeg
end

In [ ]:
function ψPosTen(ε_in,tag)
    λ, μ, k, GcI, GcII, ψ_Crit_I, ψ_Crit_II, σcI, σcII= material_tag(tag)
    εdev = I4_dev ⊙ ε_in
    εPos, εNeg = Eigen(εdev) 
    if tr(ε_in) > 0.0
        ψPlus = 0.5*k*(tr(ε_in))^2
    else
        ψPlus = 0.0
    end
    return ψPlus
end

In [ ]:
function ψPosShear(ε_in,tag)
    λ, μ, k, GcI, GcII, ψ_Crit_I, ψ_Crit_II, σcI, σcII= material_tag(tag)
    εdev = I4_dev ⊙ ε_in
    εPos, εNeg = Eigen(εdev) 
    ψPlus = μ * (εPos ⊙ εPos)
    return ψPlus
end

In [ ]:
function new_EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_in = ψhPos_in
  if ψPlus_in >= ψPlusPrev_in
    ψPlus_out = ψPlus_in
  else
    ψPlus_out = ψPlusPrev_in
  end
  true,ψPlus_out
end

In [ ]:
function EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_out = ψhPos_in
  true,ψPlus_out
end

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = Gridap.solve(op)
    return  qh
end

In [ ]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["BottomEdge","TopEdge"],
          dirichlet_masks=[(true,true), (false,true)])

uh = zero(V0_Disp)

In [ ]:
degree = 2*order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree) 

In [ ]:
LoadTagId = get_tag_from_name(labels,"TopEdge")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
nls = NLSolver(
  show_trace=true,
  method=:newton,
  linesearch=BackTracking(), iterations = 10)
solver = FESolver(nls)

In [ ]:
function run_PF(x0,vApp,cache,tag,ψTenPrev_in,ψShearPrev_in)
     λ, μ, k, GcI, GcII, ψ_Crit_I, ψ_Crit_II, σcI, σcII= material_tag(tag)
     U_PF = TrialFESpace(V0_PF)
     res(s,ϕ) = ∫((2/pi)*ls*∇(ϕ) ⋅ ∇(s) + (degDer1(s,GcI))*(ψTenPrev_in/GcI)*ϕ + (degDer2(s,GcII))*(ψShearPrev_in/GcII)*ϕ + ((1/pi)*(2-2*s)/ls)*ϕ)*dΩ
     op = FEOperator(res,U_PF,V0_PF)
     sh_out = FEFunction(U_PF,x0)
     sh_out, cache = solve!(sh_out,solver,op,cache)
  return sh_out, get_free_dof_values(sh_out), cache
end

In [ ]:
function  stepDisp(uh_in,sh_in,vApp,tag)
    uApp1(x) = VectorValue(0.0,vApp)
    uApp2(x) = VectorValue(0.0,0.0)
    U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2])
    a_Disp(u,v) = ∫( (ε(v)⊙(σ_fun∘(ε(u),ε(uh_in),sh_in,tag))))*dΩ
     b_Disp(v) = 0.0
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,U_Disp ,V0_Disp)
    uh_out = Gridap.solve(op_Disp)
    return  uh_out
end
     

In [ ]:
function χFun1(ψhTenPos_in,ψhShearPos_in)
    χVal = (ψhTenPos_in/(ψhShearPos_in + ψhTenPos_in))^2
    return χVal
end

function χFun2(ψhTenPos_in,ψhShearPos_in)
    χVal = (ψhShearPos_in/(ψhShearPos_in + ψhTenPos_in))^2
    return χVal
end

In [ ]:
function GcPrevI(χVal1, tag)
    if tag == brick
        GcI = GcI1; σcI = σcI1
    elseif tag == mortar
        GcI = GcI2; σcI = σcI2
    end
    GcII = α * GcI
    σcII = β * σcI
    Gc = GcII + (GcI - GcII)*χVal1
    return Gc
end

function GcPrevII(χVal2, tag)
    if tag == brick
        GcI = GcI1; σcI = σcI1
    elseif tag == mortar
        GcI = GcI2; σcI = σcI2
    end
    GcII = α * GcI
    σcII = β * σcI
    Gc = GcI + (GcII - GcI)*χVal2
    return Gc
end

In [ ]:
function ψCrit_I_Prev(χValPrev1,tag)
    if tag == brick
        E =E1; ν = v1; σcI = σcI1
    elseif tag == mortar
        E =E2; ν = v2; σcI = σcI2
    end
    σcII = β * σcI
    λ = E * ν / ((1 + ν) * (1 - 2 * ν))
    μ = E / (2 * (1 + ν))
    k = λ + μ
    ψ_Crit_I = σcI^2 / (2 * k)
    return χValPrev1*ψ_Crit_I
end
function ψCrit_II_Prev(χValPrev2,tag)
    if tag == brick
        E =E1; ν = v1; σcI = σcI1
    elseif tag == mortar
        E =E2; ν = v2; σcI = σcI2
    end
    σcII = β * σcI
    λ = E * ν / ((1 + ν) * (1 - 2 * ν))
    μ = E / (2 * (1 + ν))
    k = λ + μ
    ψ_Crit_II = σcII^2 / (2 * μ)
    return χValPrev2*ψ_Crit_II
end


In [ ]:
# ============================ SIMULATION PARAMETERS ============================
vApp = 0.0
const vAppMax = 0.015
delv = 0.01 / 50
innerMax = 10
count = 0
Load = Float64[]
Displacement = Float64[]
push!(Load, 0.0)
push!(Displacement, 0.0)
# ============================ INITIAL MATERIAL FIELDS ============================
χValPrev1 = 0.5
χValPrev2 = 0.5

χVal1 = CellState(χValPrev1, dΩ)
χVal2 = CellState(χValPrev2, dΩ)

GcPrevI_field = x -> GcPrevI(χVal1(x), mat_tags(x))
GcPrevII_field = x -> GcPrevII(χVal2(x), mat_tags(x))

GcIV = project(GcPrevI_field, model, dΩ, order)
GcIIV = project(GcPrevII_field, model, dΩ, order)

ψTenPrev = ψCrit_I_Prev(χValPrev1, mat_tags)
ψShearPrev = ψCrit_II_Prev(χValPrev2, mat_tags)

x0_PF = zeros(Float64, num_free_dofs(V0_PF))
cache_1 = nothing
uh = zero(V0_Disp)
sh = zero(V0_PF)
# ============================ MAIN LOAD INCREMENT LOOP ============================
while vApp .< vAppMax
    count = count .+ 1
    delv = vApp <= 0.006 ? 0.006 / 40 : 5e-6
    vApp += delv
    println("\n>>> Displacement Step $count: vApp = $(round(vApp, digits=6))")
    # ============================ INNER PHASE-FIELD ITERATION ============================
    for inner = 1:innerMax
        ψhTenPrev = project(ψTenPrev, model, dΩ, order)
        ψhShearPrev = project(ψShearPrev, model, dΩ, order)
        # Compute residual-based relative error
        num = ∫(((2 / π) * ls * ∇(sh) ⋅ ∇(sh) +
                 (degDer1∘(sh, GcIV)) * (ψhTenPrev / GcIV) * sh +
                 (degDer2∘(sh, GcIIV)) * (ψhShearPrev / GcIIV) * sh -
                 (2 / (π * ls)) * sh^2)) * dΩ
        den = ∫((2 / (π * ls)) * sh) * dΩ
        RelErr = abs(sum(num)) / abs(sum(den))
        println("    Inner Iteration $inner | Relative Error = $(round(RelErr, digits=6))")
        # Solve phase-field
        sh, x0_PF, cache_1 = run_PF(x0_PF, vApp, cache_1, mat_tags, ψhTenPrev, ψhShearPrev)
        # Solve displacement field
        uh = stepDisp(uh, sh, vApp, mat_tags)
        # Compute updated energy contributions
        ψhTenPos = ψPosTen∘(ε(uh), mat_tags)
        ψhShearPos = ψPosShear∘(ε(uh), mat_tags)
        update_state!(new_EnergyState, ψTenPrev, ψhTenPos)
        update_state!(new_EnergyState, ψShearPrev, ψhShearPos)
        # Compute mode mixity factors
        χVal1 = χFun1∘(ψTenPrev, ψShearPrev)
        χVal2 = χFun2∘(ψTenPrev, ψShearPrev)
        # Update fracture parameters
        GcIV = GcFunI(χVal1, mat_tags)
        GcIIV = GcFunII(χVal1, mat_tags)
        ψCrit_I_in, ψCrit_II_in = ψCritFun(χVal1, χVal2, mat_tags)
        update_state!(new_EnergyState, ψTenPrev, ψCrit_I_in)
        update_state!(new_EnergyState, ψShearPrev, ψCrit_II_in)
        # Break if converged
        if RelErr < 1e-4 || isnan(RelErr)
            println("    Converged at Inner Iteration $inner")
            break
        end
    end
    # ============================ POSTPROCESSING AND OUTPUT ============================
    GcVal1 = project(GcIV, model, dΩ, order)
    GcVal2 = project(GcIIV, model, dΩ, order)
    χValC1 = project(χVal1, model, dΩ, order)
    χValC2 = project(χVal2, model, dΩ, order)
    # Reaction force at top edge
    Node_Force = sum(∫(n_Γ_Load ⋅ (σfun∘(ε(uh), ε(uh), sh, GcVal1, GcVal2))) * dΓ_Load)
    push!(Load, Node_Force[2])
    push!(Displacement, vApp)
    if mod(count, 20) == 0
        writevtk(Ω, "Modified_BKCriteria_Shear_SingleEdgeNotchPlate$count",
            cellfields = ["uh" => uh,
                          "s" => sh,
                          "epsi" => ε(uh),
                          "modefactorI" => χValC1,
                          "modefactorII" => χValC2])
    end
end